# How to pass private state between nodes

Oftentimes, you may want nodes to be able to pass state to each other that should NOT be part of the main schema of the graph. This is often useful because there may be information that is not needed as input/output (and therefore doesn't really make sense to have in the main schema) but is ABSOLUTELY needed as part of the intermediate working logic.

We will have a separate node for each step. We will only have the `question` and `answer` on the overall state. However, we will need separate states for the `search_query` and the `documents` - we will pass these as private state keys. See the conceptual docs [here](https://langchain-ai.github.io/langgraph/concepts/low_level/#multiple-schemas) for more details.

Let's look at an example!

## Setup

First, let's install the required packages

In [1]:
%%capture --no-stderr
%pip install -U langgraph

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>    

## Define and use the graph

In [54]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# The overall state of the graph (this is the public state shared across nodes)
class OverallState(TypedDict):
    a: str

# Output from foo contains private data that is not part of the overall state
class FooOutput(TypedDict):
    private_data: str

# The private data is only shared between foo and bar
def foo(state: OverallState) -> FooOutput:
    output = {"private_data": "set by foo"}
    print(f'Entered node `foo`:\n\tInput: {state}.\n\tReturned: {output}')
    return output


# Step 2 (bar) input includes both the overall state and the private data from foo
class BarInput(OverallState, FooOutput):
    pass

def bar(state: BarInput) -> OverallState:
    output = {"a": "set by bar"}
    print(f'Entered node `bar`:\n\tInput: {state}.\n\tReturned: {output}')
    return output

# Buzz only has access to the overall state (no access to private data from foo)
def buzz(state: OverallState) -> OverallState:
    output = {"a": "set by buzz"}
    print(f'Entered node `buzz`:\n\tInput: {state}.\n\tReturned: {output}')
    return output


# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(foo),  # foo is the first node
builder.add_node(bar),  # bar is the second node and accepts private data from foo
builder.add_node(buzz), # buzz is the third node and does not see the private data
builder.add_edge(START, "foo")  # Start the graph with foo
builder.add_edge("foo", "bar")  # Pass from foo to bar
builder.add_edge("bar", "buzz") # Pass from bar to buzz (only overall state is shared)
builder.add_edge("buzz", END)   # End the graph after buzz
graph = builder.compile()

# Invoke the graph with the initial state
response = graph.invoke({
    "a": "set at start", 
})

print()
print(f"Output of graph invocation: {response}")

Entered node `foo`:
	Input: {'a': 'set at start'}.
	Returned: {'private_data': 'set by foo'}
Entered node `bar`:
	Input: {'a': 'set at start', 'private_data': 'set by foo'}.
	Returned: {'a': 'set by bar'}
Entered node `buzz`:
	Input: {'a': 'set by bar', 'private_data': 'set by foo'}.
	Returned: {'a': 'set by buzz'}

Output of graph invocation: {'a': 'set by buzz'}
